In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/Users/fernando.carazo/git-repos/skforecast


In [2]:
import os
# os.environ["KERAS_BACKEND"] = "torch"
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import pandas as pd

from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model

# set backend torch
import torch
import keras
print(keras.__version__)
print(keras.backend.backend())



3.10.0
tensorflow


# Test with Ximo

In [3]:
from skforecast.datasets import fetch_dataset
data = fetch_dataset('bike_sharing_extended_features')

end_train = '2012-03-31 23:59:00'
end_validation = '2012-08-31 23:59:00'
data_train = data.loc[: end_train, :]
data_val = data.loc[end_train:end_validation, :]
data_test = data.loc[end_validation:, :]


bike_sharing_extended_features
------------------------------
Hourly usage of the bike share system in the city of Washington D.C. during the
years 2011 and 2012. In addition to the number of users per hour, the dataset
was enriched by introducing supplementary features. Addition includes calendar-
based variables (day of the week, hour of the day, month, etc.), indicators for
sunlight, incorporation of rolling temperature averages, and the creation of
polynomial features generated from variable pairs. All cyclic variables are
encoded using sine and cosine functions to ensure accurate representation.
Fanaee-T,Hadi. (2013). Bike Sharing Dataset. UCI Machine Learning Repository.
https://doi.org/10.24432/C5W894.
Shape of the dataset: (17352, 90)


# Without Exog

In [4]:
lags = 24
steps = 36
levels = ["users"]
activation = "relu"
optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.MeanSquaredError()
recurrent_units = 100
dense_units = [128, 64]

model = create_and_compile_model(
    series=data[levels],
    lags=lags,
    steps=steps,
    levels=levels,
    # exog=data[exog_features],
    recurrent_units=recurrent_units,
    dense_units=dense_units,
    activation=activation,
    optimizer=optimizer,
    loss=loss,
)

model.summary()

# ==============================================================================

forecaster = ForecasterRnn(
    regressor=model,
    levels=levels,
    fit_kwargs={
        "epochs": 10,  # Number of epochs to train the model.
        "batch_size": 2048,  # Batch size to train the model.
    },
)    


# Backtesting model with exogenous variables on test data 
# # ============================================================================== 
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster_multiseries
 
cv = TimeSeriesFold( steps = 36, initial_train_size = len(data[:end_validation]), refit = False, allow_incomplete_fold=False)
metric, predictions = backtesting_forecaster_multiseries(
    forecaster = forecaster, 
    series = data[levels], 
    # exog = data[exog_features], 
    cv = cv, 
    metric = 'mean_absolute_error'
    )

metric

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ series_input (InputLayer)       │ (None, 24, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 36, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 36, 128)        │        12,928 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 36, 64)         │         8,256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 36, 1)          │            65 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,049 (242.38 KB)

 Trainable params: 62,049 (242.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


/Users/fernando.carazo/git-repos/skforecast/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Users/fernando.carazo/git-repos/skforecast/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 255ms/step - loss: 0.1028
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 294ms/step - loss: 0.0347
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 290ms/step - loss: 0.0335
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 269ms/step - loss: 0.0320
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 281ms/step - loss: 0.0318
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 280ms/step - loss: 0.0309
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 281ms/step - loss: 0.0298
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 277ms/step - loss: 0.0304
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 300ms/step - loss: 0.0293
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 292ms/step - loss: 0.0287


100%|██████████| 80/80 [00:02<00:00, 39.99it/s]


,levels,mean_absolute_error
0,users,165.376471


# With Exog Features

In [5]:
exog_features = [ 'month_sin', 'month_cos', 'week_of_year_sin', 'week_of_year_cos', 'week_day_sin', 'week_day_cos', 'hour_day_sin', 'hour_day_cos', 'sunrise_hour_sin', 'sunrise_hour_cos', 'sunset_hour_sin', 'sunset_hour_cos', 'holiday_previous_day', 'holiday_next_day', 'temp_roll_mean_1_day', 'temp_roll_mean_7_day', 'temp_roll_max_1_day', 'temp_roll_min_1_day', 'temp_roll_max_7_day', 'temp_roll_min_7_day', 'temp', 'holiday' ]


In [6]:
lags = 24
steps = 36
levels = ["users"]
activation = "relu"
# optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.MeanSquaredError()
recurrent_units = 100
dense_units = [128, 64]

model = create_and_compile_model(
    series=data[levels],
    lags=lags,
    steps=steps,
    levels=levels,
    exog=data[exog_features],
    recurrent_units=recurrent_units,
    dense_units=dense_units,
    activation=activation,
    # optimizer=optimizer,
    loss=loss,
)

model.summary()

# ==============================================================================

forecaster = ForecasterRnn(
    regressor=model,
    levels=levels,
    fit_kwargs={
        "epochs": 10,  # Number of epochs to train the model.
        "batch_size": 2048,  # Batch size to train the model.
    },
)    


# Backtesting model with exogenous variables on test data 
# # ============================================================================== 
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster_multiseries
 
cv = TimeSeriesFold( steps = 36, initial_train_size = len(data[:end_validation]), refit = False, allow_incomplete_fold=False)
metric, predictions = backtesting_forecaster_multiseries( forecaster = forecaster, series = data[levels], exog = data[exog_features], cv = cv, metric = 'mean_absolute_error' )

metric

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ series_input        │ (None, 24, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 100)       │     40,800 │ series_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 36, 100)   │          0 │ lstm_1[0][0]      │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ exog_input          │ (None, 36, 22)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 36, 122)   │          0 │ repeat_vector_1[… │
│ (Concatenate)       │                   │            │ exog_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 36, 128)   │     15,744 │ concatenate[0][0] │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_4  │ (None, 36, 64)    │      8,256 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_5  │ (None, 36, 1)     │         65 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 64,865 (253.38 KB)

 Trainable params: 64,865 (253.38 KB)

 Non-trainable params: 0 (0.00 B)

/Users/fernando.carazo/git-repos/skforecast/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 302ms/step - loss: 0.1589
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 308ms/step - loss: 0.0353
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 285ms/step - loss: 0.0314
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 288ms/step - loss: 0.0279
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 296ms/step - loss: 0.0235
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 308ms/step - loss: 0.0198
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 327ms/step - loss: 0.0177
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 309ms/step - loss: 0.0173
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 304ms/step - loss: 0.0172
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 308ms/step - loss: 0.0166


100%|██████████| 80/80 [00:02<00:00, 36.17it/s]


,levels,mean_absolute_error
0,users,134.60786
